In [ ]:
# Cell 2: Define tools and start MCP server (TCP) in background
import asyncio
import anyio
import json
from datetime import datetime
from typing import Any, Dict

import nest_asyncio
nest_asyncio.apply()

from rich import print as rprint
from fastmcp import MCPServer, Tool

# ---- Tool handlers with simple logging ----
async def add_tool(params: Dict[str, Any]):
    ts = datetime.utcnow().isoformat()
    a = params.get("a")
    b = params.get("b")
    rprint(f"[{ts}] [bold cyan]TOOL add[/] called with a={a}, b={b}")
    if not isinstance(a, (int, float)) or not isinstance(b, (int, float)):
        err = "a and b must be numbers"
        rprint(f"[{ts}] [bold red]ERROR[/] {err}")
        return {"error": err}
    res = a + b
    rprint(f"[{ts}] [bold green]RESULT[/] add -> {res}")
    return {"result": res}

async def multiply_tool(params: Dict[str, Any]):
    ts = datetime.utcnow().isoformat()
    a = params.get("a")
    b = params.get("b")
    rprint(f"[{ts}] [bold cyan]TOOL multiply[/] called with a={a}, b={b}")
    if not isinstance(a, (int, float)) or not isinstance(b, (int, float)):
        err = "a and b must be numbers"
        rprint(f"[{ts}] [bold red]ERROR[/] {err}")
        return {"error": err}
    res = a * b
    rprint(f"[{ts}] [bold green]RESULT[/] multiply -> {res}")
    return {"result": res}

# ---- Build and register tools ----
server = MCPServer(name="mcp-calculator")

server.add_tool(
    Tool(
        name="add",
        description="Add two numbers",
        input_schema={
            "type": "object",
            "properties": {
                "a": {"type": "number"},
                "b": {"type": "number"}
            },
            "required": ["a", "b"],
            "additionalProperties": False
        },
        handler=add_tool,
    )
)

server.add_tool(
    Tool(
        name="multiply",
        description="Multiply two numbers",
        input_schema={
            "type": "object",
            "properties": {
                "a": {"type": "number"},
                "b": {"type": "number"}
            },
            "required": ["a", "b"],
            "additionalProperties": False
        },
        handler=multiply_tool,
    )
)

# ---- Start server over TCP (good for notebooks) ----
SERVER_HOST = "127.0.0.1"
SERVER_PORT = 8765

server_task = None

async def start_server():
    # If your fastmcp version doesn't have serve_tcp, check server.run_stdio() alternative below.
    await server.serve_tcp(SERVER_HOST, SERVER_PORT)

async def run_server_bg():
    async with anyio.create_task_group() as tg:
        await tg.spawn(start_server)

loop = asyncio.get_event_loop()
if 'server_task' in globals() and server_task:
    rprint("[yellow]Server already running[/]")
else:
    server_task = loop.create_task(run_server_bg())
    rprint(f"[green]Server starting on tcp://{SERVER_HOST}:{SERVER_PORT}[/]")
